# Tapering off qubits using spatial symmetries

The following code demonstrates the procedure to taper off qubits using point group symmetries present in the molecules. This notebook is part of the repository mole_geo_symmetries hosted on github. The repository is compatible only with qiskit-0.11.0 
There are three main files that the following code uses:

1. r_mat_for_mols.py: This file is for running the pyscf for getting the integrals for a given molecule and the R matrices for the various symmetries present in the molecule.

2. taper_qubits_rm_funcs.py: This is the file with all the subroutines for tapering off the qubits using the point group symmetries. This file also contains a script that runs through the whole procedure for a given molecule. Part of this script is used in this jupyter notebook to demonstrate the procedure.

3. int_func.py: This is a helper file which is a modified version of the qiskit driver for pyscf. This was developed to get the atomic integrals.

The code below is structure as follows:
- import statements
- setup for a given molecule and getting an object that contains R matrices, fermionicoperator. The R matrices and fermionic operators are used to get the V matrix.
- The fermionicoperator is transformed using Jordan-Wigner mapping to qubit operator(qub_op). A copy of fermionicoperator is transformed using the v matrix. Another qubit operator (v_qubit_op) is obtained using Jordan-Wigner mapping on the transformed Hamiltonian.
- Exact diagonalization is performed on the qub_op to get the reference eigenvalues.
- The symmetries are constructed from the transformed R-matrices.
- As a sanity check, it is verified that these symmetries commute with the Hamiltonian.
- The cliffords are obtained and the qubits are tapered off. This is an iterative process as we do not know the correct eigen sector.
- The correct tapered off qubit operator is printed along with the lowest eigenvalues.
- Finally, the fermionic operator and qubit operator are obtained in the molecular orbital basis and the lowest eigenvalue are obtained as another sanity check for the values obtained from the AO basis.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# importing stuff
import logging
import copy
import itertools
from qiskit.quantum_info import Pauli
from pyscf import gto, scf, ao2mo
from pyscf.lib import param
from scipy import linalg as scila
from pyscf.lib import logger as pylogger
from qiskit.chemistry import QMolecule
import numpy as np
from qiskit.aqua.operators import WeightedPauliOperator as Operator
# from qiskit.aqua.operators import weighted_pauli_operator as Operator
from qiskit.aqua.algorithms import ExactEigensolver
import scipy
from pyscf.scf.hf import get_ovlp
from symmetries import find_symmetry_ops
from qiskit.chemistry import FermionicOperator
from int_func import qmol_func
logger = logging.getLogger(__name__)
import int_func
from int_func import qmol_func
from taper_qubits_rm_funcs import *
from r_mat_for_mols import mol_r_matrices, check_commute
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)


# AO basis setup

In [ ]:
# Change the following flag to true to print the R matrices.
# The order of the 
FLAG_PRINT_R_MATRICES = False
AO = True
# R matrices have been coded for, 'H2', 'H2O_l', 'H2O', 'NH3', 'BeH2', 'C2H2', 'C2H4', 'LiH'
molecule = 'BeH2'
# The following flag is for checking if all the r matrices commute with the Hamiltonian.
# All the r matrices in the file r_mat_for_mols_old .py have been checked. If new r_matrices are added 
# then the flag could be turned to True.
check_r_mat_commut=False
# This calculates the reference eigenspectrum. Make the following
# flag true, only when the system is small.
check_ref_energy = True
# r_mat_func 
x = r_mat_funcs(molecule, check_r_mat_commut,AO)
# printing the R-matrices for BeH2
counter = 1
if FLAG_PRINT_R_MATRICES:
    for i in x.r_matrices:
        print('R'+str(counter)+' = ')
        print(i)
        counter+=1
# The R-matrices are simultaneously diagonalized
[r_mat_evals,v_matrix] = x.sim_diag(x.r_matrices)


In [ ]:
# Getting the qubit opertor form of Hamiltonian
qub_op = x.fer_op.mapping('jordan_wigner')
# Total number of terms in the qubit operator Hamiltonian
qub_op.chop()
print('Number of terms in the Hamiltonian in AO basis')
print(len(qub_op._paulis))

# Getting and transforming the Ham with V matrix.
v_qubit_op = x.sym_transf_ham_qub_op(v_matrix)

#Printing the number of terms in the Hamiltonian
print('Final number of terms in the Hamiltonian in AO basis after the transformation')
print(len(v_qubit_op._paulis))

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# Sanity check. Calculate the spectrum of Hamiltonian
if check_ref_energy ==True:
    ee = ExactEigensolver(qub_op, k=3)
    ee_result = ee.run()
#   Get the first three eigenvalues of the Hamiltonian spectrum
    ref_min_eigvals = ee_result['eigvals'][0:3]
    # This is the reference value from Hamiltonian in AO basis
    print('Eigen value of the full Ham in AO basis')
    print(ref_min_eigvals)

# The symmetries are not independent, so, the following code gets the independent set of symmetries.
r_mat_evals = x.ind_symm_r_ev_mat(r_mat_evals)
sym_list = x.get_symm_list(r_mat_evals)


In [ ]:
# In order to check if the symmetries commute with Ham, uncomment the following piece of code:
print("check the commutativity of the found symmetry paulis between H'.")
for symm in sym_list:
    symm_op = Operator(paulis=[[1.0, symm]])
    is_commutes = check_commute(symm_op, v_qubit_op)
    print(symm_op.print_details())
    # symm_op.to_matrix()
    # print('Trace of the operators')
    # print(np.trace(symm_op._matrix.todense()))
    sym_la = symm.to_label()[::-1]
    ind = [i for i, a in enumerate(sym_la) if a == 'Z']
    print(ind)
    print("symmetry {} commutes with the Hamiltonian.".format("" if is_commutes else "NOT"))

# exit()
# Get the unitary operators (cliffords) corresponding the single qubit string.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from qiskit.aqua.operators.legacy import Z2Symmetries
[cliffords, single_qubit_list, sq_pauli_list] = x.get_cliffords(r_mat_evals,sym_list)
print('Following are the qubits which are tappered off.')
print(single_qubit_list)

z2symm = Z2Symmetries(sym_list, sq_pauli_list, single_qubit_list)
# the_tapered_ops = z2symm.taper(qub_op)


print("Trying to tapering")
correct_sector = None
for taper_coeff in itertools.product([1, -1], repeat=len(single_qubit_list)):
#     tapered_qubit_op = Operator.qubit_tapering(v_qubit_op, cliffords, single_qubit_list, list(taper_coeff))
    tapered_qubit_op = z2symm.taper(v_qubit_op, list(taper_coeff))
    ee = ExactEigensolver(tapered_qubit_op, k=1)
    ee_result = ee.run()
    temp_min_eigvals = ee_result['eigvals'][0]
    if np.isclose(temp_min_eigvals, ref_min_eigvals[0], rtol=1e-8):
        correct_sector = list(taper_coeff)
    print("at sector {}: eig value: {}; reference: {}".format(list(taper_coeff), temp_min_eigvals, ref_min_eigvals.real))

# Get the tappered qubit operator
tapered_qubit_op = x.get_tapered_qubit_op(v_qubit_op,cliffords,single_qubit_list,correct_sector)
ee = ExactEigensolver(tapered_qubit_op.copy(), k=6)
ee_result = ee.run()
print('Getting the eigen values of the tappered off qubit operator')
print(ee_result['eigvals'][0:6])


# Molecular orbital basis


In [ ]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.chemistry import QMolecule
import warnings
from qiskit.chemistry.fermionic_operator import FermionicOperator
warnings.filterwarnings("ignore",category=DeprecationWarning)

FLAG_PRINT_R_MATRICES = False
AO = True
# R matrices have been coded for, 'H2', 'H2O_l', 'H2O', 'NH3', 'BeH2', 'C2H2', 'C2H4', 'LiH'
molecule = 'BeH2'
# The following flag is for checking if all the r mat/rices commute with the Hamiltonian.
# All the r matrices in the file r_mat_for_mols_old .py have been checked. If new r_matrices are added 
# then the flag could be turned to True.
check_r_mat_commut=False
# This calculates the reference eigenspectrum. Make the following
# flag true, only when the system is small.
check_ref_energy = True
# r_mat_func 
x = r_mat_funcs(molecule, check_r_mat_commut,AO)
qub_op = x.fer_op.mapping('jordan_wigner')
# Total number of terms in the qubit operator Hamiltonian
qub_op.chop()

ee = ExactEigensolver(qub_op, k=6)
ee_result = ee.run()
ref_min_eigvals = ee_result['eigvals'][0:6]
print(ref_min_eigvals)